# TOC entitlements mapped

In [6]:
import geopandas as gpd
import intake
import pandas as pd

import ipyleaflet
from ipyleaflet import Map, GeoData, LayersControl, basemaps, WidgetControl
from ipywidgets import link, FloatSlider, Text, HTML
from ipywidgets.embed import embed_minimal_html
import branca.colormap
import json

catalog = intake.open_catalog("../catalogs/*.yml")

In [7]:
# Merge parcels with TOC/non-TOC entitlements to tracts
def make_tract_level():
    df = catalog.toc_parcels_with_entitlements.read()

    crosswalk_parcels_tracts = catalog.crosswalk_parcels_tracts.read()

    census = catalog.census_tracts.read()
    census = (census[['GEOID10', 'geometry']]
              .rename(columns = {'GEOID10':'GEOID'})
             ).to_crs("EPSG:4326")
    
    gdf1 = pd.merge(df, crosswalk_parcels_tracts, 
                   on = ["AIN", "TOC_Tier"], how = "inner", validate = "m:1")

    # Aggregate to tract-level
    gdf2 = (gdf1.groupby('GEOID')
           .agg({'AIN': 'count', 
                 'num_TOC': 'sum', 
                 'num_nonTOC': 'sum'})
           .sort_values('GEOID')
           .reset_index()
          )

    # Add geometry back in to map
    gdf3 = pd.merge(census, gdf2, 
                     on = "GEOID", how = "left", validate = "1:1")
           
    gdf3 = gdf3.assign(
        AIN = gdf3.AIN.fillna(0).astype(int),
        num_TOC = gdf3.num_TOC.fillna(0).astype(int),
        num_nonTOC = gdf3.num_nonTOC.fillna(0).astype(int),
        num = 0
    )
    
    return gdf3

In [8]:
gdf = make_tract_level()
gdf.head()

,GEOID,geometry,AIN,num_TOC,num_nonTOC,num
0,06037297602,"MULTIPOLYGON (((-118.28798 33.72258, -118.2879...",65,0,65,0
1,06037297601,"MULTIPOLYGON (((-118.27738 33.70783, -118.2773...",0,0,0,0
2,06037293306,"MULTIPOLYGON (((-118.30783 33.77937, -118.3078...",0,0,0,0
3,06037295103,"MULTIPOLYGON (((-118.30189 33.76051, -118.3018...",0,0,0,0
4,06037980015,"MULTIPOLYGON (((-118.30892 33.77497, -118.3064...",0,0,0,0


In [9]:
geo_data = json.loads(gdf.set_index('GEOID').to_json())

# Take what we want to map and turn it into a dictionary
# Can only include the key-value pair, the value you want to map, nothing more.
df = dict(zip(gdf['GEOID'].tolist(), gdf['num_TOC'].tolist()))

census_df = dict(zip(gdf['GEOID'].tolist(), gdf['num'].tolist()))

toc_max = gdf.num_TOC.max()

In [10]:
m = ipyleaflet.Map(center = (34.0536, -118.2427), zoom = 10,
                  basemap = basemaps.CartoDB.Positron)

census_tracts = ipyleaflet.Choropleth(
    geo_data = geo_data,
    choro_data = census_df, 
    colormap = branca.colormap.LinearColormap([
        "#FFFFFF",
        "#FFFFFF",        
    ]),
    border_color = '#999999',
    style = {'fillOpacity': 0.4, 'weight': 0.5, 'color': '#999999', 'opacity': 0.8},
    value_min = 0,
    value_max = 0,
    name = 'Census Tracts'
)

toc = ipyleaflet.Choropleth(
    geo_data = geo_data,
    choro_data = df,
    colormap = branca.colormap.LinearColormap([
        "#C9FFFD",
        "#06BEE1",
        "#1768AC",
        "#2541B2",
        "#03256C"
    ]),
    border_color = 'white',
    style = {'fillOpacity': 0.8, 'weight': 0.5, 'color': 'white', 'opacity': 0.8},
    hover_style = {'fillOpacity': 0.85},
    value_min = 1,
    value_max = toc_max,
    name = 'TOC entitlements'
)

plot_col = ["num_TOC"]

html = HTML(''' 
    Hover over a tract
''')

html.layout.margin = '0 px 10px 10px 10px'

def click_handler(event = None, id = None, properties = None):
    label.value = properties[plot_col]

def update_html_tract(feature, id, **kwargs): 
    html.value = '''
        Census Tract:  
        <b>{}</b> <br>
        Number of TOC Entitlements (2017-2019):
        {} 
    '''.format(id, feature['properties'][plot_col])    
    
toc.on_click(click_handler) 
toc.on_hover(update_html_tract)

m.add_layer(census_tracts)
m.add_layer(toc)

control = ipyleaflet.WidgetControl(widget = html, position = 'topright')
layers_control = ipyleaflet.LayersControl(position = 'topright')

m.add_control(control)
m.add_control(layers_control)

m.layout.height = '100%'
m.layout.min_height = '600px'
m

Map(center=[34.0536, -118.2427], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…